## Gaussian Processes with `pyro`

### Setting, training and performing inference 

Let's start by creating some synthetic data for regression

In [ ]:
# Synthetic data
se = 0.1
np.random.seed(0)
x = np.linspace(0, 1, num=20) #100x1
x_test = np.linspace(-0.4, 1.6, num=200).astype('float32')
f = lambda x : x*np.sin(10*x)

x = np.delete(x, slice(9, 14))
y = f(x) + se*np.random.randn(len(x))
fig, ax = plt.subplots(figsize=(7, 3), tight_layout=True)
ax.scatter(x, y);

x_torch = torch.from_numpy(x.astype('float32'))
y_torch = torch.from_numpy(y.astype('float32'))

We will use [`pyro.contrib.gp`](http://docs.pyro.ai/en/stable/contrib.gp.html) to implement our first GP

Let's start by creating a kernel from `gp.kernels`

We will use a Radial Basis Function (RBF) aka Squared Exponential aka Gaussian kernel as our covariance

We can specify the initial value of the variance and the lengthscale

In [ ]:
import pyro.contrib.gp as gp

pyro.enable_validation(True)
pyro.set_rng_seed(0)

K = gp.kernels.RBF(input_dim=1, 
                   variance=torch.tensor(1.), 
                   lengthscale=torch.tensor(0.01))

How does this model looks before fitting the data? 

Let's inspect the prior $p(f) = \mathcal{N}(0, K)$ on the test data

**Activity:** Increase/decrese the lengthscale and repeat, get a notion of its influence

In [ ]:
# We sum a small value to the diagonal for numerical stability
C = K.forward(torch.from_numpy(x_test)) + torch.eye(len(x_test))*1e-4
# Then we sample from the a multivariate normal distribution
samples = pyro.distributions.MultivariateNormal(torch.zeros(len(x_test)), 
                                                covariance_matrix=C).sample(sample_shape=(20,))
        
fig, ax = plt.subplots(figsize=(6, 3))
for i in range(samples.shape[0]):
    ax.plot(x_test, samples.detach().numpy()[i, :],
            linestyle='-', c='tab:blue', alpha=0.5)

In [ ]:
# Helper function to train GP and plot the results

def train_gp_plots(model, x, y, x_test, nepochs=2000):
    fig, ax = plt.subplots(1, 2, figsize=(7, 3), tight_layout=True)
    line_loss = ax[1].plot([], [])
    ax[0].scatter(x, y)
    epoch_loss = np.zeros(shape=(nepochs,))

    for k in tqdm(range(len(epoch_loss))):
        optimizer.zero_grad()
        loss = criterion(model.model, model.guide)
        loss.backward()
        optimizer.step()
        epoch_loss[k] = loss.item()
        #break    
        if k % 100 == 0:
            ax[0].cla()
            # Predictions at x_test
            mu, cov = model.forward(x_test, full_cov=True, noiseless=False)
            mu = mu.detach().numpy()
            sd = cov.diag().sqrt().detach().numpy()        
            ax[0].scatter(x, y, c='k')
            ax[0].plot(x_test.detach(), mu)
            ax[0].fill_between(x_test.detach(), mu-2*sd, mu+2*sd, alpha=0.5)
            line_loss[0].set_xdata(range(k))
            line_loss[0].set_ydata(epoch_loss[:k])
            ax[1].relim()
            ax[1].autoscale_view()
            fig.canvas.draw()

Then we create a model from `gp.models`. For regression there is `GPRegression` and for classification (non-gaussian likelihoods) we can use `VariationalGP`. There are also more efficient (sparse) versions of both models

The model expects 

- the train data
- the kernel 
- initial value of the noise variance

We may also specify a mean function for the GP

To train the model we have to select an optimizer and a cost function. We will use Adam and the Trace_ELBO, respectively

Training is very similar to how we train neural networks in pytorch

In [ ]:
pyro.clear_param_store()

#Kernel
K = gp.kernels.RBF(input_dim=1, 
                   variance=torch.tensor(1.0), 
                   lengthscale=torch.tensor(0.01))
# Model
gpr_model = gp.models.GPRegression(x_torch, y_torch, # Training data
                                   mean_function=None, # Mean
                                   kernel=K, # Covarianze
                                   jitter=1e-6, # Increase this if you have numerical problems 
                                   noise=torch.tensor(2.) # The variance of the white noise
                                   )
# Optimizer
optimizer = torch.optim.Adam(gpr_model.parameters(), lr=1e-2)
# Criterion
criterion = pyro.infer.Trace_ELBO().differentiable_loss

train_gp_plots(gpr_model, x, y, torch.from_numpy(x_test))

The learned parameters are

In [ ]:
display("RBF variance:", gpr_model.kernel.variance.item())
display("RBF length scale:", gpr_model.kernel.lengthscale.item())
display("Noise variance:", gpr_model.noise.item())

To do predictions we use the forward attribute of our `GPRegression` instance

In [ ]:
# We sum a small value to the diagonal for numerical stability
mu, Sigma = gpr_model.forward(torch.from_numpy(x_test), full_cov=True, noiseless=True)
Sigma += torch.eye(len(x_test))*1e-5
# Then we sample from the a multivariate normal distribution
samples = pyro.distributions.MultivariateNormal(mu, covariance_matrix=Sigma).sample(sample_shape=(50,))
        
fig, ax = plt.subplots(figsize=(6, 3))
for i in range(samples.shape[0]):
    ax.plot(x_test, samples.detach().numpy()[i, :], 
            linestyle='-', c='tab:blue', alpha=0.25)
ax.scatter(x, y, c='k', zorder=100);

### Trying different kernels

Kernel are implemented in [pyro.contrib.gp.kernels](http://docs.pyro.ai/en/stable/contrib.gp.html#module-pyro.contrib.gp.kernels)

Compare the RBF and Matern52 kernels. What differences do you observe?

In [ ]:
pyro.clear_param_store()

#Kernel
#K = gp.kernels.RBF(input_dim=1, variance=torch.tensor(1.0), lengthscale=torch.tensor(0.1))
K = gp.kernels.Matern52(input_dim=1, variance=torch.tensor(1.0), lengthscale=torch.tensor(0.1))
#K = gp.kernels.Periodic(input_dim=1, variance=torch.tensor(2.0), 
#                        lengthscale=torch.tensor(0.1), 
#                        period=torch.tensor(2.))
# Model
gpr_model = gp.models.GPRegression(x_torch, y_torch, 
                                   kernel=K, 
                                   noise=torch.tensor(2.))
# Optimizer
optimizer = torch.optim.Adam(gpr_model.parameters(), lr=1e-2)
# Criterion
criterion = pyro.infer.Trace_ELBO().differentiable_loss
# Train and plot
train_gp_plots(gpr_model, x, y, torch.from_numpy(x_test))

### Setting priors for the parameters 

Before we did an MLE-like estimation to find point estimates of the kernel hyper-parameters and the noise variance

We can "go bayesian" and treat these parameters as random variables and set priors for them

Training with these priors is equivalent to the MAP solution

In [ ]:
pyro.clear_param_store()
from pyro.distributions import LogNormal

#Kernel
K = gp.kernels.RBF(input_dim=1, variance=torch.tensor(1.0), lengthscale=torch.tensor(0.1))
# Model
gpr_model_prior = gp.models.GPRegression(x_torch, y_torch, 
                                   kernel=K, 
                                   noise=torch.tensor(2.))

# Setting priors
gpr_model_prior.kernel.lengthscale = pyro.nn.PyroSample(LogNormal(0.0, 1.0))
gpr_model_prior.kernel.variance = pyro.nn.PyroSample(LogNormal(0.0, 1.0))

# Optimizer
optimizer = torch.optim.Adam(gpr_model_prior.parameters(), lr=1e-2)
# Criterion
criterion = pyro.infer.Trace_ELBO().differentiable_loss
# Train and plot    
train_gp_plots(gpr_model_prior, x, y, torch.from_numpy(x_test))

### Combining kernels

Adding or multiplying valid kernels yield a valid kernel

> We can easily create new kernels from other kernels

and take advantage of their different properties

The following data has a periodic oscilation on a rising trend:

In [ ]:
x = np.random.rand(100).astype('float32')*100
y = (0.03*x + np.sin(0.1*x) + 0.1*np.random.randn(100)).astype('float32')

Try to fit it using `K1`, `K2`, `Ksum12` and `Kprod12`

Can you explain in simple words your results?

In [ ]:
pyro.clear_param_store()

K1 = gp.kernels.Periodic(input_dim=1, variance=torch.tensor(1.), 
                        lengthscale=torch.tensor(10),
                        period=torch.tensor(50))
K2 = gp.kernels.Linear(input_dim=1, variance=torch.tensor(1.))
Ksum12 = gp.kernels.Sum(K1, K2)
Kprod12 = gp.kernels.Product(K1, K2)

# Model
gpr_model = gp.models.GPRegression(torch.from_numpy(x), torch.from_numpy(y), 
                                   kernel=K1, noise=torch.tensor(2.))

optimizer = torch.optim.Adam(gpr_model.parameters(), lr=1e-2)
criterion = pyro.infer.Trace_ELBO().differentiable_loss
train_gp_plots(gpr_model, x, y, torch.linspace(-50, 150, 100), nepochs=1000)

### Sparse Gaussian Processes with `pyro`

Fitting a Gaussian process has cubic complexity

Sparse Gaussian processes use a much smaller set of "inducing points" to compute the kernel

In [ ]:
# Synthetic data
se = 0.1
np.random.seed(0)
x = np.linspace(0, 1, num=1000) #100x1
x_test = np.linspace(-0.1, 1.1, num=200).astype('float32')
f = lambda x : x*np.sin(10*x)

x = np.delete(x, slice(9, 14))
y = f(x) + se*np.random.randn(len(x))

x_torch = torch.from_numpy(x.astype('float32'))
y_torch = torch.from_numpy(y.astype('float32'))

In [ ]:
pyro.clear_param_store()

#Kernel
K = gp.kernels.RBF(input_dim=1, 
                   variance=torch.tensor(1.0), 
                   lengthscale=torch.tensor(0.1))
# Model
gpr_model = gp.models.GPRegression(x_torch, y_torch, 
                                   kernel=K, 
                                   noise=torch.tensor(2.))
#gpr_model = gp.models.SparseGPRegression(x_torch, y_torch, approx='VFE',
#                                         kernel=K, Xu=torch.linspace(0, 1, 20),
#                                         noise=torch.tensor(2.), jitter=1e-5)
# Optimizer
optimizer = torch.optim.Adam(gpr_model.parameters(), lr=1e-2)
# Criterion
criterion = pyro.infer.Trace_ELBO().differentiable_loss

train_gp_plots(gpr_model, x, y, torch.from_numpy(x_test))

## Topics for the future

- Example of GP for classification (Bernoulli likelihood), Barber 19.5
- Deep Gaussian Processes/Stacks of Gaussian Processes. Example in Pyro: https://fehiepsi.github.io/blog/deep-gaussian-process/
- [Compositional kernel learning](https://arxiv.org/pdf/1806.04326.pdf)
- Bayesian optimization with Gaussian processes
- https://github.com/team-approx-bayes/dnn2gp

## Self-study

- Mackay's book, chapter 45 on Gaussian Proceses
- Barber's book, chapter 19 on Gaussian Processes
- [Rasmussen & Willams, "Gaussian Process for Machine Learning"](http://gaussianprocess.org/gpml/?)
